## <a name="C4">0. Importation et fonctions</a>

In [36]:
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

# Feature engineering
import math
import datetime
import pandas as pd
import numpy as np
from feature_engine.imputation import MeanMedianImputer
from feature_engine.encoding import RareLabelEncoder, OneHotEncoder, OrdinalEncoder, MeanEncoder, DecisionTreeEncoder, CountFrequencyEncoder
from feature_engine.discretisation import EqualFrequencyDiscretiser
from feature_engine.transformation import YeoJohnsonTransformer, ArcsinTransformer, LogTransformer
from feature_engine.outliers import Winsorizer, OutlierTrimmer, ArbitraryOutlierCapper
from feature_engine import selection
from feature_engine.pipeline import Pipeline
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
from sklearn import feature_selection

# Model
from sklearn import model_selection, preprocessing, metrics
from sklearn.model_selection import GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_log_error, make_scorer, root_mean_squared_log_error, root_mean_squared_error


In [37]:
def get_df_from_array(pipe, X_train, X_test):
    cols = pipe.get_feature_names_out()
    X_train = pd.DataFrame(X_train, columns=cols)
    X_test = pd.DataFrame(X_test, columns=cols)

    return X_train, X_test 

In [38]:
def print_pipeline_results(grid_search):
    results = grid_search.cv_results_
    mean_r2 = results['mean_test_r2']
    mean_rmse = results['mean_test_rmse']
    model_names = []
    for model in results['params']:
        model = model['model']
        model_names.append(type(model).__name__)

    for i in range(len(param_grid)):
        print(f"{model_names[i]} mean R2: {mean_r2[i]} mean RMSE: {mean_rmse[i]}")

In [39]:
def invert_yeojhonson(value, lmbda):
  if value>= 0 and lmbda == 0:
    return exp(value) - 1
  elif value >= 0 and lmbda != 0:
    return (value * lmbda + 1) ** (1 / lmbda) - 1
  elif value < 0 and lmbda != 2:
    return 1 - (-(2 - lmbda) * value + 1) ** (1 / (2 - lmbda))
  elif value < 0 and lmbda == 2:
    return 1 - exp(-value)

## <a name="C4">1. Préparation du dataset</a>

### 1.1 Chargement du dataset

In [40]:
# Load data
df = pd.read_csv('data/clean_dataset.csv')

print(df.shape)

df.head()

(1428, 27)


,Unnamed: 0,BuildingType,PrimaryPropertyType,ZipCode,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,...,ENERGYSTARScore,SiteEnergyUse(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity,MissingENERGYSTARScore
0,0,NonResidential,Hotel,98101.0,7,DOWNTOWN,47.61220,-122.33799,1927,1.0,...,60.0,7226362.5,2003882.00,3946027.0,1276453.0,Compliant,No,249.98,2.83,False
1,1,NonResidential,Hotel,98101.0,7,DOWNTOWN,47.61317,-122.33393,1996,1.0,...,61.0,8387933.0,0.00,3242851.0,5145082.0,Compliant,No,295.86,2.86,False
2,2,NonResidential,Hotel,98101.0,7,DOWNTOWN,47.61393,-122.33810,1969,1.0,...,43.0,72587024.0,21566554.00,49526664.0,1493800.0,Compliant,No,2089.28,2.19,False
3,3,NonResidential,Hotel,98101.0,7,DOWNTOWN,47.61412,-122.33664,1926,1.0,...,56.0,6794584.0,2214446.25,2768924.0,1811213.0,Compliant,No,286.43,4.67,False
4,4,NonResidential,Hotel,98121.0,7,DOWNTOWN,47.61375,-122.34047,1980,1.0,...,75.0,14172606.0,0.00,5368607.0,8803998.0,Compliant,No,505.01,2.88,False


### 1.2 Création & suppression de variables

Je crée les variables contenant de l'information que j'estime accessible en conditions réelles, évitant la fuite de données.

In [41]:
# Creating energetical mix variables

energy_sources_vars = ['SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)']

for var in energy_sources_vars:
    var_name = var.replace('(kBtu)', '') + '_share'
    df[var_name] = df[var] / df['SiteEnergyUse(kBtu)']
    print(f'Mean {var_name} : {df[var_name].mean()}')

Mean SteamUse_share : 0.02158003255645078
Mean Electricity_share : 0.7066826149453451
Mean NaturalGas_share : 0.26833055018443125


In [42]:
# Building age

df['BuildingAge'] = datetime.date.today().year - df['YearBuilt']

Je supprime les variables inutiles, ou qui ne seront pas accessibles à ce modèle en conditions réelles.  
Je garde les variables de consommation d'énergies spécifiques pour l'instant, puisqu'elles me serviront lors de l'étape de création de features.

In [43]:
# Drop unused/unavailable variables
unused_vars = ['Unnamed: 0', 'TotalGHGEmissions', 'GHGEmissionsIntensity'] + energy_sources_vars
df = df.drop(columns=unused_vars)

In [44]:
target_var = 'SiteEnergyUse(kBtu)'

In [45]:
df = df.loc[df[target_var] != 0]

### 1.3 Gestion des types de données

Je m'assure que le type de données des variables est bien aligné avec ce que je veux en faire (hors encoding).

In [46]:
df.dtypes

BuildingType                  object
PrimaryPropertyType           object
ZipCode                      float64
CouncilDistrictCode            int64
Neighborhood                  object
Latitude                     float64
Longitude                    float64
YearBuilt                      int64
NumberofBuildings            float64
NumberofFloors                 int64
PropertyGFATotal               int64
PropertyGFAParking             int64
PropertyGFABuilding(s)         int64
ListOfAllPropertyUseTypes     object
LargestPropertyUseType        object
LargestPropertyUseTypeGFA    float64
ENERGYSTARScore              float64
SiteEnergyUse(kBtu)          float64
ComplianceStatus              object
Outlier                       object
MissingENERGYSTARScore          bool
SteamUse_share               float64
Electricity_share            float64
NaturalGas_share             float64
BuildingAge                    int64
dtype: object

Les variables de zones géographiques n'ayant pas de réel sens quantitatif, je les convertis en variables catégorielles, pour appliquer un Feature Engineering adapté.

In [47]:
num_to_cat_features = ['ZipCode', 'CouncilDistrictCode']
df[num_to_cat_features] = df[num_to_cat_features].astype('object')

Les variables booléennes ne sont pas acceptées par les modèles de ML; je les convertis en format numérique.

In [48]:
bool_to_num_features = ['MissingENERGYSTARScore']
df[bool_to_num_features] = df[bool_to_num_features].astype('object')

In [49]:
df.dtypes

BuildingType                  object
PrimaryPropertyType           object
ZipCode                       object
CouncilDistrictCode           object
Neighborhood                  object
Latitude                     float64
Longitude                    float64
YearBuilt                      int64
NumberofBuildings            float64
NumberofFloors                 int64
PropertyGFATotal               int64
PropertyGFAParking             int64
PropertyGFABuilding(s)         int64
ListOfAllPropertyUseTypes     object
LargestPropertyUseType        object
LargestPropertyUseTypeGFA    float64
ENERGYSTARScore              float64
SiteEnergyUse(kBtu)          float64
ComplianceStatus              object
Outlier                       object
MissingENERGYSTARScore        object
SteamUse_share               float64
Electricity_share            float64
NaturalGas_share             float64
BuildingAge                    int64
dtype: object

Je peux maintenant catégoriser mes variables pour faciliter les manipulations à venir.

In [50]:
# Feature vs target vars
X = df.drop(target_var, axis=1)
y = df[target_var]

In [51]:
# Numerical vs categorical features
num_features = list(X.select_dtypes(include=['int64', 'float64']).columns)
cat_features = list(set(X.columns) - set(num_features))

print("Features numériques:", num_features)
print("Features catégorielles:", cat_features)

Features numériques: ['Latitude', 'Longitude', 'YearBuilt', 'NumberofBuildings', 'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking', 'PropertyGFABuilding(s)', 'LargestPropertyUseTypeGFA', 'ENERGYSTARScore', 'SteamUse_share', 'Electricity_share', 'NaturalGas_share', 'BuildingAge']
Features catégorielles: ['ComplianceStatus', 'PrimaryPropertyType', 'MissingENERGYSTARScore', 'CouncilDistrictCode', 'Outlier', 'Neighborhood', 'LargestPropertyUseType', 'ZipCode', 'ListOfAllPropertyUseTypes', 'BuildingType']


In [52]:
# Continuous vs discrete features
discrete_features = ['NumberofFloors', 'NumberofBuildings', 'ENERGYSTARScore', 'YearBuilt', 'BuildingAge']
continuous_features = list(set(num_features) - set(discrete_features))

print("Features continues:", continuous_features)
print("Features discrètes:", discrete_features)

Features continues: ['Longitude', 'SteamUse_share', 'NaturalGas_share', 'PropertyGFATotal', 'PropertyGFAParking', 'LargestPropertyUseTypeGFA', 'Latitude', 'PropertyGFABuilding(s)', 'Electricity_share']
Features discrètes: ['NumberofFloors', 'NumberofBuildings', 'ENERGYSTARScore', 'YearBuilt', 'BuildingAge']


In [53]:
# Proportion vs interval vars
prop_features = [x for x in num_features if 'share' in x]
non_prop_features = list(set(num_features) - set(prop_features))

print("Features proportionnelles:", prop_features)
print("Features non proportionnelles:", non_prop_features)

Features proportionnelles: ['SteamUse_share', 'Electricity_share', 'NaturalGas_share']
Features non proportionnelles: ['Longitude', 'NumberofBuildings', 'YearBuilt', 'PropertyGFATotal', 'PropertyGFAParking', 'LargestPropertyUseTypeGFA', 'Latitude', 'ENERGYSTARScore', 'NumberofFloors', 'BuildingAge', 'PropertyGFABuilding(s)']


### 1.4 Split du jeu de données

Etape obligatoire avant d'apporter des modifications au jeu de données; La séparation en un jeu d'entraînement et un jeu de test.

In [54]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y,
                                                                    test_size=0.2,
                                                                    random_state=10
                                                                   )

In [55]:
X_train.shape

(1142, 24)

In [56]:
X_test.shape

(286, 24)

In [57]:
X_train_raw = X_train
y_train_raw = y_train
X_test_raw = X_test
y_test_raw = X_test

## <a name="C4">2. Feature Engineering</a>

### 2.1 Valeurs manquantes

In [58]:
# Missing values
df.isnull().sum()

BuildingType                 0
PrimaryPropertyType          0
ZipCode                      0
CouncilDistrictCode          0
Neighborhood                 0
Latitude                     0
Longitude                    0
YearBuilt                    0
NumberofBuildings            0
NumberofFloors               0
PropertyGFATotal             0
PropertyGFAParking           0
PropertyGFABuilding(s)       0
ListOfAllPropertyUseTypes    0
LargestPropertyUseType       0
LargestPropertyUseTypeGFA    0
ENERGYSTARScore              0
SiteEnergyUse(kBtu)          0
ComplianceStatus             0
Outlier                      0
MissingENERGYSTARScore       0
SteamUse_share               0
Electricity_share            0
NaturalGas_share             0
BuildingAge                  0
dtype: int64

Peu de valeurs manquantes à traiter. J'appliquerai une simple imputation par la médiane.

### 2.2 Distribution & outliers

In [59]:
# Distribution of continuous variables

fig = make_subplots(rows=2, cols=5, subplot_titles=continuous_features)

for i, col in enumerate(continuous_features, 1):
    trace = go.Histogram(x=df[col], showlegend=False)
    fig.add_trace(trace, row=(i-1)//5 + 1, col=(i-1)%5 + 1)

fig.update_layout(height=len(continuous_features)*100, title_text="Distribution of numerical features")
fig.show()

In [60]:
skewed_features = ['LargestPropertyUseTypeGFA', 'PropertyGFABuilding(s)', 'PropertyGFAParking', 'PropertyGFATotal']

In [61]:
# Distribution of discrete features

fig = make_subplots(rows=1, cols=5, subplot_titles=discrete_features)

for i, col in enumerate(discrete_features, 1):
    trace = go.Histogram(x=df[col], showlegend=False, histnorm='percent')
    fig.add_trace(trace, row=(i-1)//5 + 1, col=(i-1)%5 + 1)

fig.update_layout(height=len(discrete_features)*100, title_text="Distribution of discrete features in %")
fig.show()

In [62]:
skewed_features += ['NumberofFloors', 'NumberofBuildings']

Certaines features numériques ont une distribution très asymétrique, influencée par la présence des gros batîments collectifs identifiés dans l'exploration de données.  
Je vais appliquer des transformations mathématiques pour lisser leurs distributions, voire du capping sur les outliers si ils rendent l'erreur du modèle trop instable.

### 2.3 Cardinalité & labels rares

In [63]:
# Cardinality of categorical variables
fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Bar(y=cat_features, x=df[cat_features].nunique().values, orientation='h', showlegend=False), row=1, col=1)

rare_labels = []
for var in cat_features:
    value_counts = df[var].value_counts(normalize=True)
    rare_labels.append(len(value_counts[value_counts < 0.03]))
fig.add_trace(go.Bar(x=rare_labels, y=cat_features, orientation='h', showlegend=False), row=2, col=1)

fig.update_xaxes(title_text="Number of modalities", row=1, col=1)
fig.update_xaxes(title_text="Number of rare labels", row=2, col=1)
fig.update_layout(height=500, width=750)

fig.show()

Certaines features catégorielles possèdent une haute cardinalité et de nombreuses modalités peu fréquentes.  
Je vais donc appliquer un regroupement de ces modalités rares, avant de réaliser l'encodage de toutes les features catégorielles.

### 2.4 Transformations indépendentes du modèles

Certaines étapes du Feature engineering sont particuliérement utilisées et performantes sur certains types de modèle.  
Ici, j'applique dans un premier temps les transformations agnostiques du modèle, qui bénéficient à l'ensemble/la majorité des modèles de Machine Learning.

In [64]:
pipe = Pipeline(
    [
        ('imputer_num', MeanMedianImputer(imputation_method='median')),
        ('rare_label_encoding', RareLabelEncoder(variables=cat_features)),
        ('categorical_encoding', OneHotEncoder(variables=cat_features)),
    ]
)

In [65]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('imputer_num', MeanMedianImputer()),
                ('rare_label_encoding',
                 RareLabelEncoder(variables=['ComplianceStatus',
                                             'PrimaryPropertyType',
                                             'MissingENERGYSTARScore',
                                             'CouncilDistrictCode', 'Outlier',
                                             'Neighborhood',
                                             'LargestPropertyUseType',
                                             'ZipCode',
                                             'ListOfAllPropertyUseTypes',
                                             'BuildingType'])),
                ('categorical_encoding',
                 OneHotEncoder(variables=['ComplianceStatus',
                                          'PrimaryPropertyType',
                                          'MissingENERGYSTARScore',
                                          'CouncilDistrictCode', 'Outlier',
                                          'Neighborhood',
                                          'LargestPropertyUseType', 'ZipCode',
                                          'ListOfAllPropertyUseTypes',
                                          'BuildingType']))])

In [66]:
X_train = pipe.transform(X_train)
X_test = pipe.transform(X_test)

In [67]:
# Transform X_train array back to a DataFrame
X_train, X_test = get_df_from_array(pipe, X_train, X_test)

In [68]:
X_train.shape

(1142, 60)

### 2.5 Sélection de features indépendente du modèle

Dans la même logique, certaines étapes de la sélection de features sont agnostiques du modèle utilisé.

Ici, j'élimine les variables constantes et quasi constantes, puisqu'elles auraient peu de valeur prédictive.

In [69]:
X_train.shape

(1142, 60)

In [70]:
transformer = selection.DropConstantFeatures(tol=0.95)

transformer.fit(X_train)
X_train = transformer.transform(X_train)
X_test = transformer.transform(X_test)

X_train.shape

(1142, 57)

In [71]:
transformer.features_to_drop_

['ComplianceStatus_Compliant', 'Outlier_No', 'BuildingType_NonResidential']

In [72]:
transformer = selection.DropDuplicateFeatures()

transformer.fit(X_train)
X_train = transformer.transform(X_train)
X_test = transformer.transform(X_test)

X_train.shape

(1142, 57)

J'ai pu réduire la dimensionalité en supprimant quelques features doublon, ou avec peu de variance, n'altérant pas la qualité de mon modèle.

## <a name="C4">3. Modélisation linéaire</a>

### 3.1 Feature Engineering spécifique

Les modèles linéaires nécéssitent souvent un engineering spécifique, notamment sur:
- **La distribution**, les modèles travaillant sous l'hypothèse de distributions normales et étant sensibles aux outliers
- **La magnitude**, pour ne pas biaiser le poids de chaque prédicteur

In [73]:
X_train_lin = X_train
y_train_lin = y_train

In [74]:
tf = YeoJohnsonTransformer(variables=target_var)
y_train_lin = pd.DataFrame(y_train_lin)
tf.fit(y_train_lin)
y_train_lin = tf.transform(y_train_lin)
y_test_lin = pd.DataFrame(y_test)
y_test_lin = tf.transform(y_test_lin)

In [75]:
y_train_lin

,SiteEnergyUse(kBtu)
1212,8.739526
1144,8.922522
334,9.126922
467,9.161771
565,8.731679
...,...
1393,9.278016
1344,9.061864
527,9.346661
1149,8.659001


In [76]:
temp = [feature for feature in num_features if feature in X_train_lin.columns]
num_features = temp

In [77]:
distrib_tf = YeoJohnsonTransformer(variables=num_features)

pipe = Pipeline(
    [
        # ('outlier trimming', OutlierTrimmer(variables=num_features, capping_method='gaussian', fold=3)),
        ('distribution_transformation', distrib_tf),
        ('scaler', preprocessing.StandardScaler())
    ]
)

In [78]:
pipe.fit(X_train_lin, y_train_lin)

Pipeline(steps=[('distribution_transformation',
                 YeoJohnsonTransformer(variables=['Latitude', 'Longitude',
                                                  'YearBuilt',
                                                  'NumberofBuildings',
                                                  'NumberofFloors',
                                                  'PropertyGFATotal',
                                                  'PropertyGFAParking',
                                                  'PropertyGFABuilding(s)',
                                                  'LargestPropertyUseTypeGFA',
                                                  'ENERGYSTARScore',
                                                  'SteamUse_share',
                                                  'Electricity_share',
                                                  'NaturalGas_share',
                                                  'BuildingAge'])),
                ('scaler', StandardScaler())])

In [79]:
X_train_lin  = pipe.transform(X_train_lin)
X_test_lin = pipe.transform(X_test)
# X_train_lin, y_train_lin = pipe.transform_x_y(X_train, y_train)
# X_test_lin, y_test_lin = pipe.transform_x_y(X_test, y_test)

In [80]:
# Transform X_train array back to a DataFrame
X_train_lin, X_test_lin = get_df_from_array(pipe, X_train_lin, X_test_lin)

In [81]:
X_train_lin.shape

(1142, 57)

In [82]:
X_train_lin

,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),LargestPropertyUseTypeGFA,ENERGYSTARScore,...,ZipCode_Rare,ZipCode_98122.0,ZipCode_98108.0,ZipCode_98104.0,ZipCode_98101.0,ZipCode_98105.0,ListOfAllPropertyUseTypes_Rare,"ListOfAllPropertyUseTypes_Office, Parking",ListOfAllPropertyUseTypes_Office,ListOfAllPropertyUseTypes_Non-Refrigerated Warehouse
0,1.387779e-16,-0.363561,0.518342,0.060368,0.203841,-0.795536,-0.546966,-0.704165,-0.752262,-0.444716,...,-0.751255,-0.239598,-0.283578,-0.334629,-0.331384,-0.231313,0.564541,-0.304669,-0.308088,-0.274462
1,-4.440892e-16,0.233331,0.091341,0.060368,-1.171825,-0.994788,-0.546966,-0.893029,-0.938120,0.247399,...,-0.751255,-0.239598,-0.283578,-0.334629,-0.331384,-0.231313,0.564541,-0.304669,-0.308088,-0.274462
2,1.110223e-16,-0.331213,0.725195,0.060368,0.802258,1.121894,1.841218,0.753229,0.763716,1.399713,...,-0.751255,-0.239598,-0.283578,-0.334629,-0.331384,-0.231313,-1.771350,3.282255,-0.308088,-0.274462
3,-5.551115e-17,-0.437298,0.484478,0.060368,1.154957,0.997562,1.845469,-0.131885,1.008988,1.276162,...,-0.751255,-0.239598,-0.283578,-0.334629,-0.331384,-0.231313,0.564541,-0.304669,-0.308088,-0.274462
4,0.000000e+00,-0.755274,-0.653659,0.060368,0.203841,0.302639,-0.546966,0.369418,0.415577,1.462166,...,-0.751255,-0.239598,-0.283578,-0.334629,-0.331384,-0.231313,-1.771350,-0.304669,3.245821,-0.274462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1137,0.000000e+00,1.332217,1.738033,0.060368,0.997950,0.538089,-0.546966,0.608544,0.630423,-1.738729,...,1.331107,-0.239598,-0.283578,-0.334629,-0.331384,-0.231313,0.564541,-0.304669,-0.308088,-0.274462
1138,-1.665335e-16,0.264467,1.233107,0.060368,1.568694,0.369450,-0.546966,0.436888,0.365745,0.462826,...,-0.751255,-0.239598,-0.283578,2.988383,-0.331384,-0.231313,0.564541,-0.304669,-0.308088,-0.274462
1139,8.326673e-17,-0.731596,1.539909,0.060368,0.802258,0.863548,1.823198,0.741352,0.767358,-2.059651,...,-0.751255,-0.239598,-0.283578,-0.334629,-0.331384,-0.231313,0.564541,-0.304669,-0.308088,-0.274462
1140,-1.942890e-16,1.029250,-0.248350,0.060368,-0.306293,-0.893229,-0.546966,-0.796960,-0.555154,0.247399,...,1.331107,-0.239598,-0.283578,-0.334629,-0.331384,-0.231313,0.564541,-0.304669,-0.308088,-0.274462


### 3.2 Sélection et importance des features

In [83]:
X_train_transf = X_train_lin
y_train_transf = y_train_lin

Je filtre ensuite les variables trop corrélées entre elles, pour éviter un problème de multicollinéarité et réduire la dimensionalité.

In [84]:
transformer = selection.SmartCorrelatedSelection(method='pearson',
                                                 threshold=0.99,
                                                 selection_method="model_performance", 
                                                 estimator=LinearRegression(),
                                                 cv=3)

In [85]:
transformer.fit(X_train_lin, y_train_lin)
X_train_lin = transformer.transform(X_train_lin)
X_test_lin = transformer.transform(X_test_lin)
X_train_lin.shape

(1142, 54)

In [86]:
transformer.correlated_feature_sets_

[{'BuildingAge', 'YearBuilt'},
 {'LargestPropertyUseType_Hotel', 'PrimaryPropertyType_Hotel'},
 {'MissingENERGYSTARScore_False', 'MissingENERGYSTARScore_True'}]

In [87]:
transformer.features_to_drop_

['BuildingAge', 'LargestPropertyUseType_Hotel', 'MissingENERGYSTARScore_False']

J'ai pu supprimer les features extrêmement corrélées (principalement des features binaires).

Je peux maintenant utiliser le mécanisme de régularisation de la régression Lasso pour garder les features les plus importantes.  
Pour cela, j'effectue d'abord une Grid Search pour trouver le meilleur alpha à utiliser.

In [88]:
parameters = {'alpha': [1e-5, 1e-3, 1e-1, 1, 10, 1000, 10000, 100000]}

grid_search = GridSearchCV(Lasso(), parameters, cv=5, scoring='r2')

grid_search.fit(X_train_lin, y_train_lin)

best_alpha = grid_search.best_params_['alpha']
best_lasso = grid_search.best_estimator_

In [89]:
# Best alpha
print('Best alpha:', best_alpha)

# Best model score
print('Best model score (r2):', grid_search.best_score_)

Best alpha: 0.001
Best model score (r2): 0.6897134680706787


Je vais garder uniquement les features à coefficients non-nuls selon la Lasso.

In [90]:
lasso = feature_selection.SelectFromModel(Lasso(alpha=best_alpha, random_state=10))

lasso.fit(X_train_lin, y_train_lin)

SelectFromModel(estimator=Lasso(alpha=0.001, random_state=10))

In [91]:
# Keeping only non-O coefficient features
X_train_lin = X_train_lin[X_train_lin.columns[lasso.get_support()]]

In [92]:
X_test_lin = X_test_lin[X_test_lin.columns[lasso.get_support()]]

In [93]:
X_train_lin.shape

(1142, 41)

In [94]:
X_test_lin.shape

(286, 41)

Grace à la régréssion lasso et à toutes les étapes préalables de sélection de features, j'ai pu fortement réduire la dimensionalité du modèle, tout en minimisant la multi-collinéarité et améliorant sa performance.

In [95]:
regressor = Lasso(alpha=best_alpha)
r2_raw = np.mean(cross_val_score(regressor, X_train, y_train, cv=5, scoring='r2'))
rmse_raw = np.mean(cross_val_score(regressor, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error'))

In [96]:
regressor = Lasso(alpha=best_alpha)
r2 = np.mean(cross_val_score(regressor, X_train_lin, y_train_lin, cv=5, scoring='r2'))
rmse = np.mean(cross_val_score(regressor, X_train_lin, y_train_lin, cv=5, scoring='neg_root_mean_squared_error'))

In [97]:
print("Nombre de features avant sélection:", len(X_train.columns))
print("Nombre de features après sélection:", len(X_train_lin.columns))

Nombre de features avant sélection: 57
Nombre de features après sélection: 41


In [98]:
print("R2 et RMSE avant sélection:", r2_raw, rmse_raw)
print("R2 et RMSE après sélection:", r2, rmse)

R2 et RMSE avant sélection: 0.3363117665780756 -12094794.281444836
R2 et RMSE après sélection: 0.6914074883616715 -0.22909586523206213


In [99]:
X_train_lin

,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),LargestPropertyUseTypeGFA,ENERGYSTARScore,SteamUse_share,...,ZipCode_98134.0,ZipCode_98121.0,ZipCode_Rare,ZipCode_98122.0,ZipCode_98108.0,ZipCode_98101.0,ZipCode_98105.0,ListOfAllPropertyUseTypes_Rare,"ListOfAllPropertyUseTypes_Office, Parking",ListOfAllPropertyUseTypes_Non-Refrigerated Warehouse
0,-0.363561,0.518342,0.060368,0.203841,-0.795536,-0.546966,-0.704165,-0.752262,-0.444716,-0.266684,...,-0.367680,-0.247666,-0.751255,-0.239598,-0.283578,-0.331384,-0.231313,0.564541,-0.304669,-0.274462
1,0.233331,0.091341,0.060368,-1.171825,-0.994788,-0.546966,-0.893029,-0.938120,0.247399,-0.266684,...,2.719753,-0.247666,-0.751255,-0.239598,-0.283578,-0.331384,-0.231313,0.564541,-0.304669,-0.274462
2,-0.331213,0.725195,0.060368,0.802258,1.121894,1.841218,0.753229,0.763716,1.399713,-0.266684,...,-0.367680,-0.247666,-0.751255,-0.239598,-0.283578,-0.331384,-0.231313,-1.771350,3.282255,-0.274462
3,-0.437298,0.484478,0.060368,1.154957,0.997562,1.845469,-0.131885,1.008988,1.276162,-0.266684,...,-0.367680,4.037701,-0.751255,-0.239598,-0.283578,-0.331384,-0.231313,0.564541,-0.304669,-0.274462
4,-0.755274,-0.653659,0.060368,0.203841,0.302639,-0.546966,0.369418,0.415577,1.462166,-0.266684,...,-0.367680,4.037701,-0.751255,-0.239598,-0.283578,-0.331384,-0.231313,-1.771350,-0.304669,-0.274462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1137,1.332217,1.738033,0.060368,0.997950,0.538089,-0.546966,0.608544,0.630423,-1.738729,-0.266684,...,-0.367680,-0.247666,1.331107,-0.239598,-0.283578,-0.331384,-0.231313,0.564541,-0.304669,-0.274462
1138,0.264467,1.233107,0.060368,1.568694,0.369450,-0.546966,0.436888,0.365745,0.462826,-0.266684,...,-0.367680,-0.247666,-0.751255,-0.239598,-0.283578,-0.331384,-0.231313,0.564541,-0.304669,-0.274462
1139,-0.731596,1.539909,0.060368,0.802258,0.863548,1.823198,0.741352,0.767358,-2.059651,-0.266684,...,-0.367680,-0.247666,-0.751255,-0.239598,-0.283578,-0.331384,-0.231313,0.564541,-0.304669,-0.274462
1140,1.029250,-0.248350,0.060368,-0.306293,-0.893229,-0.546966,-0.796960,-0.555154,0.247399,-0.266684,...,-0.367680,-0.247666,1.331107,-0.239598,-0.283578,-0.331384,-0.231313,0.564541,-0.304669,-0.274462


### 3.3 Optimisation de la performance

Pour optimiser encore plus, je vais comparer les performances de la Lasso avec une régression Ridge, une régression linéaire et un Dummy Regressor, afin de choisir le meilleur modèle linéaire possible.

In [100]:
param_grid = [
    {'model': [DummyRegressor(strategy="mean")]},
    {'model': [LinearRegression()]},
    {'model': [Ridge(alpha=100)]},
    {'model': [Lasso(best_alpha)]},
]

In [101]:
pipe = Pipeline(steps=[('model', LinearRegression())])
grid_search = GridSearchCV(pipe, 
                           param_grid, 
                           cv=5,
                          scoring={
                              'r2': 'r2', 
                              'rmse': 'neg_root_mean_squared_error'
                          },
                           refit='r2'
                          )



In [102]:
grid_search.fit(X_train_lin, y_train_lin)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('model', LinearRegression())]),
             param_grid=[{'model': [DummyRegressor()]},
                         {'model': [LinearRegression()]},
                         {'model': [Ridge(alpha=100)]},
                         {'model': [Lasso(alpha=0.001)]}],
             refit='r2',
             scoring={'r2': 'r2', 'rmse': 'neg_root_mean_squared_error'})

In [103]:
results = grid_search.cv_results_
mean_r2 = results['mean_test_r2']
mean_rmse = results['mean_test_rmse']
model_names = []
for model in results['params']:
    model = model['model']
    model_names.append(type(model).__name__)

In [104]:
for i in range(len(param_grid)):
    print(f"{model_names[i]} mean R2: {mean_r2[i]} mean RMSE: {mean_rmse[i]}")

DummyRegressor mean R2: -0.0023515786330452924 mean RMSE: -0.41465321708113867
LinearRegression mean R2: 0.6904123977629673 mean RMSE: -0.22948443508854863
Ridge mean R2: 0.6981318861889283 mean RMSE: -0.22676590463642748
Lasso mean R2: 0.6914074883616715 mean RMSE: -0.22909586523206213


Suite à tout le travail de sélection, les performances entre les différentes régressions linéaires sont identiques.

Le modèle retenu sera donc la régression linéaire classique, pour sa simplicité.

### 3.4 Analyse de l'importance des features

In [105]:
lambda_dict = distrib_tf.lambda_dict_

Je peux maintenant analyser l'importance des différentes features.

In [106]:
regressor = LinearRegression()
regressor.fit(X_train_lin, y_train_lin)

LinearRegression()

In [107]:
df_coeffs = pd.DataFrame({'feature': X_train_lin.columns, 'coeff': regressor.coef_.round(3)[0]})

In [108]:
# Coefficient for each feature
df_coeffs = df_coeffs.sort_values(by='coeff', ascending=False)
px.bar(df_coeffs, x='coeff', y='feature', height=len(X_train_lin.columns)*30)

In [109]:
# Top 10 features in absolute coefficient
df_coeffs['abs_coeff'] = abs(df_coeffs.coeff)
df_coeffs = df_coeffs.sort_values(by='abs_coeff', ascending=False)
px.bar(df_coeffs[0:10], x='abs_coeff', y='feature', height=len(X_train.columns)*20)

## <a name="C4">4. Modélisation non linéaire</a>

Les modèles non linéaires sont moins sensibles à la magnitude et à la distribution des features, mais bénéficient d'une dimensionalité réduite.  
Je vais donc les tester sur la matrice de features ne contenant pas ces transformations, et sur celle transformée, pour évaluer l'impact.

J'ai choisi d'utiliser deux modèles non linéaires, un arbre de décision, et une Random Forest.

In [110]:
param_grid = [
    {'model': [DecisionTreeRegressor()]},
    {'model': [RandomForestRegressor()]},
]

In [111]:
pipe = Pipeline(steps=[('model', DummyRegressor())])
grid_search = GridSearchCV(pipe, 
                           param_grid, 
                           cv=5,
                          scoring={
                              'r2': 'r2', 
                              'rmse': 'neg_root_mean_squared_error'
                          },
                           refit='r2'
                          )



In [112]:
# Training on transformed data
grid_search.fit(X_train_transf, y_train_transf)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('model', DummyRegressor())]),
             param_grid=[{'model': [DecisionTreeRegressor()]},
                         {'model': [RandomForestRegressor()]}],
             refit='r2',
             scoring={'r2': 'r2', 'rmse': 'neg_root_mean_squared_error'})

In [113]:
print_pipeline_results(grid_search)

DecisionTreeRegressor mean R2: 0.42261866316640584 mean RMSE: -0.3138777305646728
RandomForestRegressor mean R2: 0.6902139816951315 mean RMSE: -0.22988722433293846


In [114]:
# Training on untransformed data
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('model', DummyRegressor())]),
             param_grid=[{'model': [DecisionTreeRegressor()]},
                         {'model': [RandomForestRegressor()]}],
             refit='r2',
             scoring={'r2': 'r2', 'rmse': 'neg_root_mean_squared_error'})

In [115]:
print_pipeline_results(grid_search)

DecisionTreeRegressor mean R2: -0.29518987888025305 mean RMSE: -16063011.777401727
RandomForestRegressor mean R2: 0.42750684834439723 mean RMSE: -11828354.840637228


Il y'a en effet peu de différence entre la performance sur le jeu de données transformé, et le jeu de donné non transformé.  
Le jeu de données transformé sera donc préféré pour sa dimensionalité réduite.

## <a name="C4">5. Modèle final</a>

Le modèle final retenu est la Random Forest, qui apporte les meilleurs résultats sur les métriques de R2 et de RMSE, en étant moins sensible aux outliers.

Grâce à une Randomized Search, on obtient la meilleure combinaison possible de tous les hyperparamètres importants.

In [116]:
X_train = X_train_lin
y_train = y_train_lin
X_test = X_test_lin
y_test = y_test_lin

In [117]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [118]:
regressor = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = regressor, 
                               param_distributions = random_grid, 
                               n_iter = 100, 
                               cv = 3, 
                               verbose=2, 
                               random_state=42, 
                               n_jobs = -1)


In [119]:
rf_random.fit(X_train, y_train[target_var].values)
print('Done')

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   0.5s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   0.5s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   0.5s
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=12

In [120]:
rf_random.best_params_

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 110,
 'bootstrap': True}

In [121]:
best_rf = rf_random.best_estimator_

Je peux finalement tester mon modèle sur le jeu de test.

In [122]:
best_rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=110, max_features='sqrt', n_estimators=1000)

In [123]:
y_test

,SiteEnergyUse(kBtu)
528,8.639957
35,8.689416
1028,8.278860
617,8.585999
483,8.477884
...,...
68,9.099504
1255,8.604616
533,9.634184
913,8.778714


In [124]:
y_pred = best_rf.predict(X_test)
r2 = r2_score(y_test, y_pred) 
rmse = root_mean_squared_error(y_test, y_pred)

In [125]:
print(r2)
print(rmse)

0.6910695875117985
0.2495200579020366


In [126]:
lambda_dict_target = tf.lambda_dict_

In [127]:
lambda_dict_target

{'SiteEnergyUse(kBtu)': -0.07481868853756102}

In [128]:
df_error = pd.DataFrame({'test': y_test[target_var].values, 'pred': y_pred})
df_error['error'] = df_error.test - df_error.pred
df_error['error_share'] = abs(df_error.error / df_error.test * 100)

In [129]:
df_error.test = df_error['test'].apply(lambda value: invert_yeojhonson(value, lambda_dict_target[target_var]))
df_error.pred = df_error['pred'].apply(lambda value: invert_yeojhonson(value, lambda_dict_target[target_var]))

In [130]:
df_error.sort_values(by='error_share')

,test,pred,error,error_share
224,4.954810e+05,4.953434e+05,0.000104,0.001246
44,3.639670e+06,3.642424e+06,-0.000244,0.002699
143,9.814820e+06,9.790760e+06,0.000736,0.007865
53,8.174096e+05,8.153047e+05,0.000931,0.010905
83,3.611538e+06,3.595802e+06,0.001411,0.015598
...,...,...,...,...
168,4.168064e+07,3.181791e+06,0.763350,7.813864
215,2.671351e+05,1.721440e+06,-0.682826,8.411320
193,3.427261e+05,2.438892e+06,-0.703348,8.561888
243,1.174384e+05,8.430919e+05,-0.765196,9.828924


In [131]:
df_error.error_share.sort_values()

224     0.001246
44      0.002699
143     0.007865
53      0.010905
83      0.015598
         ...    
168     7.813864
215     8.411320
193     8.561888
243     9.828924
61     11.014626
Name: error_share, Length: 286, dtype: float64

In [132]:
custom_bins = list(range(0, 110, 10)) + [float('inf')]
bin_labels = [f'{i}-{i+10}' for i in range(0, 100, 10)] + ['100+']

df_error['binned'] = pd.cut(df_error.error_share, bins=custom_bins, labels=bin_labels, right=False)
df_error['binned'] = df_error['binned'].astype(str)
df_error = df_error.sort_values(by='binned')

In [133]:
df_bin = pd.DataFrame(df_error.groupby('binned').size().sort_values(ascending=False)).reset_index()

In [134]:
px.histogram(df_bin, x='binned', y=0)

In [135]:
df_error.error_share.mean()

2.0638517854970004

## <a name="C4">6. Test de l'ENERGY STAR Score</a>

Nous avons déjà pu nous rendre compte à travers l'analyse de l'importance des features des modèles linéaires que l'ENERGY STAR Score était un des prédicteurs les plus importants pour la consommation d'énergie.  
Je vais chercher à confirmer cette observation à travers un A/B Test.

In [136]:
X_train_minus_star_score = X_train.drop(columns='ENERGYSTARScore')
X_test_minus_star_score = X_test.drop(columns='ENERGYSTARScore')

In [137]:
best_rf.fit(X_train_minus_star_score, y_train)

RandomForestRegressor(max_depth=110, max_features='sqrt', n_estimators=1000)

In [138]:
y_pred = best_rf.predict(X_test_minus_star_score)
r2 = r2_score(y_test, y_pred) 
rmse = root_mean_squared_error(y_test, y_pred)

In [139]:
print(r2)
print(rmse)

0.6463691973432738
0.2669624465852317


Le modèle est en effet moins performant et moins explicatif si l'on retire l'ENERGY STAR Score, mais la différence n'est pas marquée.   
Cette baisse de performance devra être pesée contre la complexité de collection de cette donnée pour décider si cette feature est conservée ou non.